In [1]:
import yadisk
import pandas as pd
from collections import defaultdict
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
# Установите ваши токены доступа
y = yadisk.YaDisk(token="y0_AgAAAAAAVtKZAAykGAAAAAEVS4ljAADxRJXH2c9ISbj80DIgMYXg-8Zhxg")

# Проверка, авторизован ли пользователь
if y.check_token():
    logger.info("Успешная авторизация")
else:
    logger.error("Ошибка авторизации")
    exit()

INFO:yadisk:sending APIRequest GetOperationStatusRequest, GET https://cloud-api.yandex.net/v1/disk/operations/0000
INFO:__main__:Успешная авторизация


In [3]:


# Функция для проверки существования пути
def path_exists(path):
    try:
        y.get_meta(path)
        return True
    except yadisk.exceptions.PathNotFoundError:
        return False

In [4]:
# dir(y.listdir('/'))
# [i for i in y.listdir('/')]

In [5]:

# Рекурсивная функция для сканирования всех файлов
def scan_folder(path="/"):
    if not path_exists(path):
        logger.error(f"Путь не найден: {path}")
        return

    for item in y.listdir(path):
        full_path = f"{path if path != '/' else ''}/{item['name']}"
        if item['type'] == 'dir':
            scan_folder(full_path)
        elif item['type'] == 'file':
            file_name = item['name']
            # full_path = item['path']
            file_size = item['size']
            # modified = item['modified']
            # resource_id = item['resource_id']
            # gps_longitude = dict(item['exif'])['gps_longitude']
            # gps_latitude = dict(item['exif'])['gps_latitude']
            file_info[(file_name, file_size)].append(full_path)
            logger.info(f"обработан: {(file_size, file_name)}")


In [ ]:

# Словарь для хранения информации о файлах
file_info = defaultdict(list)

# Начать сканирование с корневой папки
scan_folder()
# scan_folder('/Записки')

INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:yadisk:sending APIRequest GetMetaRequest, GET https://cloud-api.yandex.net/v1/disk/resources
INFO:__main__:обработан: (3561186, '190й завод_05.08.2008 12-14-53.jpg')
INFO:__main__:обработан: (3588738, '190й завод_05.08.2008 14-52-17.jpg')
INFO:__main__:обработан: (3626906, '190й завод_05.08.2008 15-34-17.jpg

In [1]:
file_info

NameError: name 'file_info' is not defined

In [ ]:
# Список для хранения данных о дублях
duplicates = []

# Проверка на дубликаты
for (file_name, file_size), paths in file_info.items():
    if len(paths) > 1:
        for path in paths:
            folder_name = path.rsplit('/', 1)[0]
            duplicates.append({
                'File Name': file_name,
                'Full Path': path,
                'Folder Name': folder_name,
                'File Size': file_size
            })

In [ ]:
duplicates

In [ ]:

# Создание DataFrame и экспорт в CSV
df = pd.DataFrame(duplicates)
df.to_csv('duplicates.csv', index=False)

print("Сканирование завершено. Результаты сохранены в duplicates.csv")

In [ ]:
df = pd.read_csv('duplicates.csv')
df.groupby(df['File Name']).count()